<a href="https://colab.research.google.com/github/nicholasfurl/Great-Courses/blob/main/Lesson11_kmean_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 #we will write our own k-means model from scratch and use it to cluster handwritten numbers from the MNIST dataset

In [1]:
#Unlike the lesson, I like to start with the main code body and then explore the 
#function definitions when I know what data they are being passed

#Data is a "bunch"
from sklearn.datasets import fetch_openml
data = fetch_openml(name='mnist_784')

#33% of data in data allocated to test data

import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.1)
X_train, X_test, y_train, y_test = train_test_split(X_test, y_test, test_size=0.33)
len(X_train)

4690

In [ ]:
def assign_data(data,centers):
  
  # n is the number of data points
  n = len(data)
  # d is the dimensionality of the data points
  d = len(data[0])
  # k is the number of clusters
  k = len(centers)
  # first, subtract the set of centers from each data point
  res = np.reshape(data,(1,n,d))-np.reshape(centers,(k,1,d))
  # sum the squared differences
  res2 = np.add.reduce(res**2,2)
  # assign each data point to its closest center
  centerids = np.apply_along_axis(np.argmin,0,res2)
  # While we're here, make a note of the loss
  loss = sum(np.apply_along_axis(np.min,0,res2))
  return(centerids, loss)